# Credits
Code adapted from: https://github.com/decentralion/tf-dev-summit-tensorboard-tutorial/blob/master/mnist.py
# Tf.summary
Module tf.summary containts tensorflow operators that output "summary" protocolbuffer format 
Examples:  
tf.summary.scalar  
tf.summary.image  
tf.summary.audio

In [1]:
import os
import os.path
import shutil
import tensorflow as tf

LOGDIR = "./mnist_demo/"
### MNIST EMBEDDINGS ###
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(train_dir=LOGDIR + "data", one_hot=True)

/Users/trungtv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_demo/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_demo/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
LABELS = os.path.join(os.getcwd(), "labels_1024.tsv")
SPRITES = os.path.join(os.getcwd(), "sprite_1024.png")

In [3]:
def conv_layer(input, size_in, size_out, name='conv'):
    with tf.name_scope(name):
        #w = tf.Variable(tf.zeros([5, 5, size_in, size_out]), name='W')
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        #b = tf.Variable(tf.zeros([size_out]), name='B')
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def fc_layer(input, size_in, size_out, name='fc'):
    with tf.name_scope(name):
        # w = tf.Variable(tf.zeros([size_in, size_out]), name='W')
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        # b = tf.Variable(tf.zeros([size_out]), name='B')
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [4]:
def mnist_model(learning_rate):
    tf.reset_default_graph()
    sess = tf.Session()
    
    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    # create the network 
    # first convec layer
    conv1 = conv_layer(x_image, 1, 32, "conv1")
    # second convec layer
    conv_out = conv_layer(conv1, 32, 64, "conv2")
    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])
    #################
    embedding_input = flattened
    #################
    # embedding_size = 7 * 7 * 64
    fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, 'fc1')
    logits = fc_layer(fc1, 1024, 10, 'fc2')
    
    #compute cross entropy as our loss function 
    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=logits, labels=y))
    tf.summary.scalar("xent", xent)

    # use and AdapOptimizer to train the network
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)
    
    # compute the accuracy
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    # Initialize all the variables 
    sess.run(tf.global_variables_initializer())
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "5-embedding-visualizer")
    writer.add_graph(sess.graph)
    
    #############
    embedding = tf.Variable(tf.zeros([1024, 7*7*64]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()
    
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = SPRITES
    embedding_config.metadata_path = LABELS
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    
    # Train for 1000 steps
    for i in range(1000):
        batch = mnist.train.next_batch(100)
        # Occasionally report accuracy 
        if i % 50 == 0:
            [train_accuracy, s] = sess.run([accuracy, merged_summary], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
            print("step %d, training accuracy %g" % (i, train_accuracy))
            ###############
            # Occationally calculate embedding for the test dataset   
            sess.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        # Run the training step
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

In [5]:
mnist_model(1e-4)

step 0, training accuracy 0.05
step 50, training accuracy 0.75
step 100, training accuracy 0.83
step 150, training accuracy 0.92
step 200, training accuracy 0.89
step 250, training accuracy 0.95
step 300, training accuracy 0.92
step 350, training accuracy 0.88
step 400, training accuracy 0.91
step 450, training accuracy 0.92
step 500, training accuracy 0.97
step 550, training accuracy 0.96
step 600, training accuracy 0.92
step 650, training accuracy 0.96
step 700, training accuracy 0.99
step 750, training accuracy 0.94
step 800, training accuracy 0.97
step 850, training accuracy 0.93
step 900, training accuracy 0.96
step 950, training accuracy 0.97


## Let visualize the Tensorflow graph
tensorboard --logdir ./mnist_demo/5-embedding-visualizer